In [1]:
from keras.models import Model

In [6]:
model = Model(inputs = x,outputs = [output1,output2])

In [3]:
from keras.layers import *

In [4]:
x = Input(shape=(3,))

In [5]:
hidden1 = Dense(128,activation='relu')(x)
hidden2 = Dense(64,activation='relu')(hidden1)

output1 = Dense(1,activation='linear')(hidden2)
output2 = Dense(1,activation='sigmoid')(hidden2)

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 3)]                  0         []                            
                                                                                                  
 dense (Dense)               (None, 128)                  512       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 64)                   8256      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 1)                    65        ['dense_1[0][0]']             
                                                                                              

In [8]:
from keras.utils import plot_model
plot_model(model,show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


**complex functional API**

In [9]:
inputA = Input(shape=(32,))
inputB = Input(shape=(128,))

In [10]:
# the first branch operates on the first input
x = Dense(8, activation="relu")(inputA)
x1 = Dense(4, activation="relu")(x)

In [11]:
# the second branch operates on second input
y = Dense(64, activation="relu")(inputB)
y1 = Dense(32, activation="relu")(y)
y2 = Dense(16, activation="relu")(y1)

In [12]:
# combine the ouputs of both branches
combined = concatenate([x1, y2])

In [13]:
# applying a fc layer and then a regression prediction on the combined layers
z = Dense(2, activation="relu")(combined)
z1 = Dense(1,activation='linear')(z)

In [14]:
model = Model(inputs=[inputA, inputB], outputs=z1)

In [15]:
plot_model(model,show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


**AGE AND GENDER USING FUNCTIONAL API**

In [16]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [17]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 98% 323M/331M [00:02<00:00, 225MB/s]
100% 331M/331M [00:02<00:00, 140MB/s]


In [18]:
import zipfile
zip = zipfile.ZipFile('/content/utkface-new.zip', 'r')
zip.extractall('/content')
zip.close()

In [19]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [20]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [21]:
age = []
gender = []
img_path = []
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(file.split('_')[1])
  img_path.append(file)

In [22]:
len(age)

23708

In [23]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [24]:
df.shape

(23708, 3)

In [25]:
df.head()

,age,gender,img
0,42,1,42_1_0_20170103184125771.jpg.chip.jpg
1,3,0,3_0_2_20170110212616630.jpg.chip.jpg
2,25,1,25_1_0_20170117143830737.jpg.chip.jpg
3,3,0,3_0_0_20170110225025587.jpg.chip.jpg
4,48,0,48_0_0_20170117160506440.jpg.chip.jpg


In [26]:
df['gender'] = df['gender'].astype('float32')

In [27]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [28]:
print(train_df.shape)
print(test_df.shape)

(20000, 3)
(3708, 3)


In [29]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [30]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [31]:
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model

In [32]:
vggnet = VGG16(include_top=False,input_shape=(200,200,3))

58889256/58889256 [==============================] - 0s 0us/step


In [33]:
vggnet.trainable=False

output = vggnet.layers[-1].output
flatten = Flatten()(output)

dense1 = Dense(512,activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [34]:
model = Model(inputs=vggnet.input,outputs=[output1,output2])

In [35]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 200, 200, 64)         1792      ['input_4[0][0]']             
                                                                                                  
 block1_conv2 (Conv2D)       (None, 200, 200, 64)         36928     ['block1_conv1[0][0]']        
                                                                                                  
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)         0         ['block1_conv2[0][0]']        
                                                                                            

In [36]:
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [37]:
model.compile(optimizer='adam',loss={'age':'mae','gender':'binary_crossentropy'},metrics={'age':'mae','gender':'accuracy'})

In [39]:
model.fit(train_generator,batch_size=32,epochs=5,validation_data=test_generator)

Epoch 1/5
625/625 [==============================] - 396s 633ms/step - loss: 11.0504 - age_loss: 10.5606 - gender_loss: 0.4898 - age_mae: 10.5606 - gender_accuracy: 0.7621 - val_loss: 9.5714 - val_age_loss: 9.2044 - val_gender_loss: 0.3670 - val_age_mae: 9.2044 - val_gender_accuracy: 0.8325
Epoch 2/5
625/625 [==============================] - 398s 637ms/step - loss: 9.7016 - age_loss: 9.2679 - gender_loss: 0.4338 - age_mae: 9.2679 - gender_accuracy: 0.7905 - val_loss: 8.5287 - val_age_loss: 8.1673 - val_gender_loss: 0.3613 - val_age_mae: 8.1673 - val_gender_accuracy: 0.8374
Epoch 3/5
625/625 [==============================] - 402s 643ms/step - loss: 9.3387 - age_loss: 8.9276 - gender_loss: 0.4111 - age_mae: 8.9276 - gender_accuracy: 0.8074 - val_loss: 8.1620 - val_age_loss: 7.8313 - val_gender_loss: 0.3306 - val_age_mae: 7.8313 - val_gender_accuracy: 0.8525
Epoch 4/5
625/625 [==============================] - 405s 648ms/step - loss: 9.1974 - age_loss: 8.7920 - gender_loss: 0.4055 - age